In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
%matplotlib inline

In [2]:
pd.set_option('display.max_rows', 200)
data = pd.read_csv("collected_data_merged.csv")
webclass_df = pd.read_csv("original_datasets/Webpages_Classification_train_data.csv")


/Users/tomasortega/Desktop/PCS_project/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (109) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
encoder = LabelEncoder()
df = data
df['file_ext'] = df['file_ext'].replace(float("nan"), "").astype("string")
df['tld'] = df['tld'].replace(float("nan"), "").astype("string")
df['dom_country'] = df['dom_country'].replace(float("nan"), "").astype("string")
df['asn_ip'] = df['asn_ip'].replace(float("nan"), "").astype("string")

df['file_ext_enc'] = encoder.fit_transform(df['file_ext'])
df['tld_enc'] = encoder.fit_transform(df['tld'])
df['dom_country_enc'] = encoder.fit_transform(df['dom_country'])
df['asn_ip_enc'] = encoder.fit_transform(df['asn_ip'])
df.drop(df[df['url'].isin(webclass_df['url'])].index, inplace=True)

In [4]:
df.sort_values('live', inplace=True)
df.drop_duplicates(['url'], keep='first', inplace=True)
df.drop(['url', 'file_ext', 'tld', 'asn_ip', 'dom_country', 'port', 'iframes'], axis=1, inplace=True)

In [5]:
# df.to_csv("phishing.csv", index=False)

In [6]:
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

Transformation pipeline

In [7]:
scaler = StandardScaler()

train_labels = train_set['label']
train_set = train_set.drop(['label'], axis=1)
df = pd.DataFrame(scaler.fit_transform(train_set), columns=train_set.columns)


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

lin_reg = LogisticRegression()
lin_reg.fit(train_set, train_labels)

/Users/tomasortega/Desktop/PCS_project/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [9]:
some_data = test_set.iloc[0:20]
print("Predictions: ", lin_reg.predict(some_data.drop(['label'], axis=1)))
print("Actual labels: ", list(some_data['label']))

Predictions:  [1 0 1 1 0 0 0 1 0 1 1 1 0 1 1 1 0 1 1 1]
Actual labels:  [1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1]


In [10]:
r_sq = lin_reg.score(train_set, train_labels)
print('coefficient of determination:', r_sq)
# scores = cross_val_score(lin_reg, train_set.drop(["label"], axis=1), train_set['label'], scoring="neg_mean_squared_error", cv=10)
# lin_rmse_score = np.sqrt(-scores)

coefficient of determination: 0.8918208551279049
